In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 60
pd.options.display.max_rows = 500
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import datetime as dt

## Forma anterior

In [2]:

# indice_precios = pd.read_csv('./data/indice_pricestats_YQ.csv', index_col=0)
# indice_precios.index = pd.DatetimeIndex(indice_precios.index)
# indice_precios.index.name = 'Q'

# indice_precios.head()


# CBAr = pd.read_csv('./data/CBA_regional.csv')
# CBAr.index = pd.date_range(start = dt.datetime(2016,4,1), end = dt.datetime(2018,12,31), freq='M')
# CBAr = CBAr.drop(['Ano4', 'Mes', 'TRIMESTRE'], axis = 1)

# CBTr = pd.read_csv('./data/CBT_regional.csv')
# CBTr.index = pd.date_range(start = dt.datetime(2016,4,1), end = dt.datetime(2018,12,31), freq='M')
# CBTr = CBTr.drop(['Ano4', 'Mes', 'TRIMESTRE'], axis = 1)

# #.stack().reset_index()
# CB = pd.concat([CBAr.stack(), CBTr.stack()], axis = 1)
# CB.columns = ['CBA', 'CBT']; CB.index.names = ['Mes', 'Region']

# # ipc = pd.read_csv('./data/Argentina_inflation_pricestats.csv', index_col=0)
# # ipc.index = pd.date_range(start = dt.datetime(1943,1,1), end = dt.datetime(2018,11,30), freq='M', name='Mes')

# # CB_ipc = CB.join(ipc, how='inner')
# # CB_ipc[['CBA','CBT']] = (CB_ipc[['CBA','CBT']].T/CB_ipc['index'].values).T
# # CB_ipc = CB_ipc.groupby([pd.Grouper(level = 0,freq = 'Q'), pd.Grouper(level = 1)]).mean()

# # idx = CB_ipc[['index']].unstack().loc['2018-09-30'][0]
# # CB_ipc = CB_ipc[['CBA', 'CBT']]*idx

# # CB_ipc = CB_ipc.loc[indice_precios.index]
# # CB_ipc.index.names = ['Q', 'Region']
# # # CB_ipc.unstack().plot(legend = False)
# # CB_ipc = CB_ipc.reset_index()

# # CB_ipc.to_csv('./data/CB_Reg_defl.csv', index = False)

## Forma actual

In [3]:

col_mon = [u'P21', u'P47T', u'PP08D1', u'TOT_P12', u'T_VI', u'V12_M', u'V2_M', u'V3_M', u'V5_M']

cpi = pd.read_csv('./data/indice_precios.csv')
cpi.index = pd.date_range("1943-01", periods=len(cpi), freq = 'M')
# cpi = cpi.groupby(pd.TimeGrouper(freq='Q')).mean().loc['2003':'2019'][['index']]#.to_csv(...)
cpi.set_index('index')
cpi.index.name = 'Q'
#
indice_precios = cpi

cpi = cpi.loc['2003-01-31': '2019-12-31']   # Actualizar aca para extender en tiempo

cpi.columns = pd.MultiIndex.from_product([['Indice de Precios'], cpi.columns])


In [4]:
CBAr = pd.read_csv('./data/CBA_regional.csv')
CBAr.index = pd.date_range(start = dt.datetime(2016,4,1), end = dt.datetime(2019,6,30), freq='M')
CBAr = CBAr.drop(['Ano4', 'Mes', 'TRIMESTRE'], axis = 1)

CBTr = pd.read_csv('./data/CBT_regional.csv')
CBTr.index = pd.date_range(start = dt.datetime(2016,4,1), end = dt.datetime(2019,6,30), freq='M')
CBTr = CBTr.drop(['Ano4', 'Mes', 'TRIMESTRE'], axis = 1)

#.stack().reset_index()
CB = pd.concat([CBAr.stack(), CBTr.stack()], axis = 1)
CB.columns = ['CBA', 'CBT']; CB.index.names = ['Mes', 'Region']

In [5]:
# # 11747 (oct-nov 2019) se tiene que convertir en 
# 11747*(2283/1906) #para junio 2019
# # 11747*(1.231485) #para 2019Q2

In [6]:

CB_i = pd.concat([CB.unstack(), cpi.iloc[:, 1]], axis = 1)
CB_defl = CB_i.iloc[:, :-1].div(CB_i.iloc[:, -1], 0)

# # Check how the fillna works...
# # fig, ax = plt.subplots(1, figsize = (7, 5))
# # CB_defl.fillna(CB_defl.mean()).plot(legend = False, ax = ax)
# # CB_defl.fillna(CB_defl.quantile(.25)).head(25).plot(legend = False, ax = ax, c = '.5')
# # CB_defl.fillna(CB_defl.quantile(.75)).head(25).plot(legend = False, ax = ax, c = '.5')

CB_defl = CB_defl.fillna(CB_defl.mean()).stack()
CB_defl_Q = CB_defl.groupby([pd.Grouper(level = 0,freq = 'Q'), pd.Grouper(level = 1)]).mean()
CB_defl_Q

# idx = CB_i.iloc[:, -1]['2019-11-30']
# CB_defl_Q = CB_defl_Q[['CBA', 'CBT']]*idx
CB_defl_Q = CB_defl_Q[['CBA', 'CBT']]*2283


CB_defl_Q.index.names = ['Q', 'Region']
CB_defl_Q = CB_defl_Q.reset_index()
CB_defl_Q[['CBA', 'CBT']] = CB_defl_Q[['CBA', 'CBT']]
CB_defl_Q.to_csv('./data/CB_Reg_defl.csv', index = False)

## Listo

In [7]:
# Debug deflacion
# val_list = []
# for q in ['2018-09-30', '2018-06-30', '2018-03-31', '2017-12-31',
#        '2017-09-30', '2017-06-30', '2017-03-31', '2016-12-31',
#        '2016-09-30', '2016-06-30']:
#     df = pd.read_csv('./data/RFReg_'+str(frac)+'ARG'+q+'.csv', 
#                      usecols = ['P47T'], nrows = 50000)
#     val_list += [df.median().values[0]]
# plt.plot(np.array(val_list))

In [8]:
CB_defl_Q.tail()

,Q,Region,CBA,CBT
403,2019-12-31,Gran Buenos Aires,4806.733774,11847.212281
404,2019-12-31,Noreste,4286.477706,9895.210260
405,2019-12-31,Noroeste,4177.908507,9560.094671
406,2019-12-31,Pampeana,4772.391837,11760.989001
407,2019-12-31,Patagónica,4948.259956,13844.020201
